In [2]:
from __future__ import print_function, division
import glob
import numpy as np
import capo
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import imageio
import os
import hsa7458_v001 as cal
from operator import itemgetter

ImportError: No module named capo

In [3]:
# code for the progress bar

import time

class ProgressBar: 
    def __init__(self, loop_length):
        import time
        self.start = time.time()
        self.increment_size = 100.0/loop_length
        self.curr_count = 0
        self.curr_pct = 0
        self.overflow = False
        print ('% complete: ',)
    
    def increment(self):
        self.curr_count += self.increment_size
        if int(self.curr_count) > self.curr_pct:
            self.curr_pct = int(self.curr_count)
            if self.curr_pct <= 100:
                print (self.curr_pct,)
            elif self.overflow == False:
                print("\n* Count has gone over 100%; likely either due to:\n  - an error in the loop_length specified when " + \
                      "progress_bar was instantiated\n  - an error in the placement of the increment() function")
                print('Elapsed time when progress bar full: {:0.1f} seconds.'.format(time.time() - self.start))
                self.overflow = True

    def finish(self):
        if 99 <= self.curr_pct <= 100: # rounding sometimes makes the maximum count 99.
            print("100",)
            print('\nElapsed time: {:0.1f} seconds.\n'.format(time.time() - self.start))
        elif self.overflow == True:
            print('Elapsed time after end of loop: {:0.1f} seconds.\n'.format(time.time() - self.start))
        else:
            print('\n* End of loop reached earlier than expected.\nElapsed time: {:0.1f} seconds.\n'.format(time.time() - self.start))

In [4]:
def calculate_baseline(antennae, pair):
    """
    The decimal module is necessary for keeping the number of decimal places small.
    Due to small imprecision, if more than 8 or 9 decimal places are used, 
    many baselines will be calculated that are within ~1 nanometer to ~1 picometer of each other.
    Because HERA's position precision is down to the centimeter, there is no 
    need to worry about smaller imprecision.
    """

    dx = antennae[pair[0]]['top_x'] - antennae[pair[1]]['top_x']
    dy = antennae[pair[0]]['top_y'] - antennae[pair[1]]['top_y']
    baseline = np.around([np.sqrt(dx**2. + dy**2.)],3)[0] #XXX this may need tuning
    slope = dy/np.float64(dx)
    if slope == -np.inf:
        slope = slope * -1
    elif slope == 0:
        slope = slope + 0

    ps = (pair[0],pair[1],"%.2f" % slope)
    return "%.1f" % baseline,ps

In [5]:
ex_ants=[]
antennae = cal.prms['antpos_ideal']
baselines = {}


for antenna_i in antennae:
    if antennae[antenna_i]['top_z'] < 0.:
        continue
    if antenna_i in ex_ants:
        continue
        
    for antenna_j in antennae:
        if antennae[antenna_j]['top_z'] < 0.:
            continue
        if antenna_j in ex_ants:
            continue

        if antenna_i == antenna_j:
            continue
        elif antenna_i < antenna_j:
            pair = (antenna_i, antenna_j)
        elif antenna_i > antenna_j:
            pair = (antenna_j, antenna_i)
        
        baseline,ps = calculate_baseline(antennae, pair)
        
        if (baseline not in baselines):
            baselines[baseline] = [ps]
        elif (pair in baselines[baseline]):
            continue
        else:
            baselines[baseline].append(ps)
            
            


NameError: name 'cal' is not defined

In [5]:
data_dir = '/home/champ10/Desktop/HERA2015/2457746/'
my_path = '/home/champ10/Desktop/Test/'
keys = sorted(baselines) 
pbar = ProgressBar(len(keys))

xx_data = sorted(glob.glob(''.join([data_dir, 'zen.*.xx.HH.uvcORR'])))
xy_data = sorted(glob.glob(''.join([data_dir, 'zen.*.xy.HH.uvcORR'])))
yx_data = sorted(glob.glob(''.join([data_dir, 'zen.*.yx.HH.uvcORR'])))
yy_data = sorted(glob.glob(''.join([data_dir, 'zen.*.yy.HH.uvcORR'])))



for i in keys:
    
    x= sorted(set(baselines[i]),key=itemgetter(2))
    print ('i=',i)
    new_folder =  'zen.2457746.testbasline{}'.format(i)
    os.makedirs(my_path + new_folder) #\t is tab in name

    
    for elem,antstr in enumerate(x):
        #print elem, antstr
        ant_i = x[elem][0]
        ant_j = x[elem][1]
        slope = x[elem][2]
        antstr1 = "%s_%s" % (x[elem][0], x[elem][1])         
         
         
         
        avg_freq_i_real = None
        avg_freq_q_real = None
        avg_freq_u_real = None
        avg_freq_v_real = None

        avg_freq_i_imag = None
        avg_freq_q_imag = None
        avg_freq_u_imag = None
        avg_freq_v_imag = None


        n_avg = 0





        for i in range(len(xx_data)):
            print (i,end=" ")
            #print("Reading {}...".format(xx_data[i]))
            t_xx, d_xx, f_xx = capo.miriad.read_files([xx_data[i]], antstr=antstr1, polstr='xx')
            #print("Reading {}...".format(xy_data[i]))
            t_xy, d_xy, f_xy = capo.miriad.read_files([xy_data[i]], antstr=antstr1, polstr='xy')
            #print("Reading {}...".format(yx_data[i]))
            t_yx, d_yx, f_yx = capo.miriad.read_files([yx_data[i]], antstr=antstr1, polstr='yx')
            #print("Reading {}...".format(yy_data[i]))
            t_yy, d_yy, f_yy = capo.miriad.read_files([yy_data[i]], antstr=antstr1, polstr='yy')

            #print("Reading {}...".format(xx_data[i]))





            vis_xx = d_xx[(ant_i, ant_j)]['xx']
            vis_yy = d_yy[(ant_i, ant_j)]['yy']
            vis_yx = d_yx[(ant_i, ant_j)]['yx']
            vis_xy = d_xy[(ant_i, ant_j)]['xy']


            stokes_I = vis_xx + vis_yy
            stokes_Q = vis_xx - vis_yy
            stokes_U = vis_xy + vis_yx
            stokes_V = 1j*vis_xy - 1j*vis_yx


            stokes_I_real = stokes_I.real
            stokes_I_imag = stokes_I.imag
            stokes_Q_real = stokes_Q.real
            stokes_Q_imag = stokes_Q.imag
            stokes_U_real = stokes_U.real
            stokes_U_imag = stokes_U.imag
            stokes_V_real = stokes_V.real
            stokes_V_imag = stokes_V.imag


    #         print ("stokes_I",stokes_I.shape)
    #         print (stokes_I[46][500:510])
    #         print (stokes_I[46][500:510].real)
    #         print (stokes_I[46][500:510].imag)


    #        print (asfa) #used as break

            # for I
            if avg_freq_i_real is None:
                avg_freq_i_real = np.zeros((vis_xx.shape[1]))
                avg_freq_i_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_q_real = np.zeros((vis_xx.shape[1]))
                avg_freq_q_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_u_real = np.zeros((vis_xx.shape[1]))
                avg_freq_u_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_v_real = np.zeros((vis_xx.shape[1]))
                avg_freq_v_imag = np.zeros((vis_xx.shape[1]))



            # loop over times
            for a in range(vis_xx.shape[0]):
                avg_freq_i_real += stokes_I_real[a, :]
                avg_freq_i_imag += stokes_I_imag[a, :]
                avg_freq_q_real += stokes_Q_real[a, :]
                avg_freq_q_imag += stokes_Q_imag[a, :]
                avg_freq_u_real += stokes_U_real[a, :]
                avg_freq_u_imag += stokes_U_imag[a, :]
                avg_freq_v_real += stokes_V_real[a, :]
                avg_freq_v_imag += stokes_V_imag[a, :]

                n_avg += 1






        # finish averaging
        avg_freq_i_real = avg_freq_i_real/n_avg
        avg_freq_i_imag = avg_freq_i_imag/n_avg
        avg_freq_q_real = avg_freq_q_real/n_avg
        avg_freq_q_imag = avg_freq_q_imag/n_avg
        avg_freq_u_real = avg_freq_u_real/n_avg
        avg_freq_u_imag = avg_freq_u_imag/n_avg
        avg_freq_v_real = avg_freq_i_real/n_avg
        avg_freq_v_imag = avg_freq_i_imag/n_avg



        np.savez(my_path+new_folder+'/'+'zen.2457747.avgstokes.{}.{}.{}.npz'.format(i,slope,antstr1),
        i_real = avg_freq_i_real,
        i_imag = avg_freq_i_imag,
        q_real = avg_freq_q_real,
        q_imag = avg_freq_q_imag,
        u_real = avg_freq_u_real,
        u_imag = avg_freq_u_imag,
        v_real = avg_freq_i_real,
        v_imag = avg_freq_i_imag)


pbar.finish()


% complete: 
i= 14.6


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 

In [1]:
baselines

NameError: name 'baselines' is not defined

In [ ]:
data_dir = '/home/champ10/Desktop/HERA2015/2457746/'
my_path = '/home/champ10/Desktop/Test/'
keys = sorted(baselines) 
pbar = ProgressBar(len(keys))

xx_data = sorted(glob.glob(''.join([data_dir, 'zen.*.xx.HH.uvcORR'])))
xy_data = sorted(glob.glob(''.join([data_dir, 'zen.*.xy.HH.uvcORR'])))
yx_data = sorted(glob.glob(''.join([data_dir, 'zen.*.yx.HH.uvcORR'])))
yy_data = sorted(glob.glob(''.join([data_dir, 'zen.*.yy.HH.uvcORR'])))


# generate antstr that has all baseline pairs in it
# e.g., antstr = '72_112,81_95,72_81'

avg_freq_i_real = None
avg_freq_q_real = None
avg_freq_u_real = None
avg_freq_v_real = None

avg_freq_i_imag = None
avg_freq_q_imag = None
avg_freq_u_imag = None
avg_freq_v_imag = None

n_avg = 0

for i in range(len(xx_data)):
    print (i,end=" ")
    print("Reading {}...".format(xx_data[i]))
    t_xx, d_xx, f_xx = capo.miriad.read_files([xx_data[i]], antstr=antstr, polstr='xx')
    print("Reading {}...".format(xy_data[i]))
    t_xy, d_xy, f_xy = capo.miriad.read_files([xy_data[i]], antstr=antstr, polstr='xy')
    print("Reading {}...".format(yx_data[i]))
    t_yx, d_yx, f_yx = capo.miriad.read_files([yx_data[i]], antstr=antstr, polstr='yx')
    print("Reading {}...".format(yy_data[i]))
    t_yy, d_yy, f_yy = capo.miriad.read_files([yy_data[i]], antstr=antstr, polstr='yy')


    for i in keys:
        x= sorted(set(baselines[i]),key=itemgetter(2))
        print ('i=',i)
        new_folder =  'zen.2457746.testbasline{}'.format(i)
        os.makedirs(my_path + new_folder) #\t is tab in name
        
        for elem,antstr in enumerate(x):
            ant_i = x[elem][0]
            ant_j = x[elem][1]
            slope = x[elem][2]

            vis_xx = d_xx[(ant_i, ant_j)]['xx']
            vis_yy = d_yy[(ant_i, ant_j)]['yy']
            vis_yx = d_yx[(ant_i, ant_j)]['yx']
            vis_xy = d_xy[(ant_i, ant_j)]['xy']

            stokes_I = vis_xx + vis_yy
            stokes_Q = vis_xx - vis_yy
            stokes_U = vis_xy + vis_yx
            stokes_V = 1j*vis_xy - 1j*vis_yx

            stokes_I_real = stokes_I.real
            stokes_I_imag = stokes_I.imag
            stokes_Q_real = stokes_Q.real
            stokes_Q_imag = stokes_Q.imag
            stokes_U_real = stokes_U.real
            stokes_U_imag = stokes_U.imag
            stokes_V_real = stokes_V.real
            stokes_V_imag = stokes_V.imag

            if avg_freq_i_real is None:
                avg_freq_i_real = np.zeros((vis_xx.shape[1]))
                avg_freq_i_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_q_real = np.zeros((vis_xx.shape[1]))
                avg_freq_q_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_u_real = np.zeros((vis_xx.shape[1]))
                avg_freq_u_imag = np.zeros((vis_xx.shape[1]))
                avg_freq_v_real = np.zeros((vis_xx.shape[1]))
                avg_freq_v_imag = np.zeros((vis_xx.shape[1]))

            # loop over times
            for a in range(vis_xx.shape[0]):
                avg_freq_i_real += stokes_I_real[a, :]
                avg_freq_i_imag += stokes_I_imag[a, :]
                avg_freq_q_real += stokes_Q_real[a, :]
                avg_freq_q_imag += stokes_Q_imag[a, :]
                avg_freq_u_real += stokes_U_real[a, :]
                avg_freq_u_imag += stokes_U_imag[a, :]
                avg_freq_v_real += stokes_V_real[a, :]
                avg_freq_v_imag += stokes_V_imag[a, :]

                n_avg += 1

# finish averaging
avg_freq_i_real = avg_freq_i_real/n_avg
avg_freq_i_imag = avg_freq_i_imag/n_avg
avg_freq_q_real = avg_freq_q_real/n_avg
avg_freq_q_imag = avg_freq_q_imag/n_avg
avg_freq_u_real = avg_freq_u_real/n_avg
avg_freq_u_imag = avg_freq_u_imag/n_avg
avg_freq_v_real = avg_freq_v_real/n_avg
avg_freq_v_imag = avg_freq_v_imag/n_avg

np.savez(my_path+new_folder+'/'+'zen.2457747.avgstokes.{}.{}.{}.npz'.format(i,slope,antstr1),
i_real = avg_freq_i_real,
i_imag = avg_freq_i_imag,
q_real = avg_freq_q_real,
q_imag = avg_freq_q_imag,
u_real = avg_freq_u_real,
u_imag = avg_freq_u_imag,
v_real = avg_freq_v_real,
v_imag = avg_freq_v_imag)


pbar.finish()
